## ***LightBoost Regressor***

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")

C:\Users\default.DESKTOP-GAN0M7C\AppData\Local\Temp\ipykernel_13744\179680575.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

### Data Cleaning & Transformation

In [5]:

def preprocess_orders(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the orders dataset:
    - Drop irrelevant or high-cardinality columns
    - Rename columns if needed
    - Extract useful features from 'Date'
    - Create derived features like day, month, weekday, weekend flag
    """
    
    # 1. Drop unwanted columns if they exist
    DROP_COLS = [
        "index",
        "Order ID",           # pure identifier
        "SKU",                # high cardinality ID
        "promotion-ids",      # sparse / leakage-prone
        "Unnamed: 22",
        "ASIN",
        "fulfilled-by",
        "ship-postal-code",
        "currency",
        "Size"
    ]
    
    data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])
    
    # 2. Rename columns if necessary
    if 'Sales Channel ' in data.columns:
        data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)
    
    # 3. Convert 'Date' to datetime safely
    if 'Date' in data.columns:
        data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")
        
        # Extract date-related features
        data["order_day"] = data["Date"].dt.day_name()
        data["order_month"] = data["Date"].dt.month_name()
        data["order_weekday"] = data["Date"].dt.weekday
        data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)
        
        # Drop original Date column
        data = data.drop(columns=["Date"])
    
    return data


In [10]:
data = preprocess_orders(data)
print(data.head())

                         Status Fulfilment Sales Channel ship-service-level  \
0                     Cancelled   Merchant     Amazon.in           Standard   
1  Shipped - Delivered to Buyer   Merchant     Amazon.in           Standard   
2                       Shipped     Amazon     Amazon.in          Expedited   
3                     Cancelled   Merchant     Amazon.in           Standard   
4                       Shipped     Amazon     Amazon.in          Expedited   

     Style       Category Courier Status  Qty  Amount    ship-city  \
0   SET389            Set            NaN    0  647.62       MUMBAI   
1  JNE3781          kurta        Shipped    1  406.00    BENGALURU   
2  JNE3371          kurta        Shipped    1  329.00  NAVI MUMBAI   
3    J0341  Western Dress            NaN    0  753.33   PUDUCHERRY   
4  JNE3671            Top        Shipped    1  574.00      CHENNAI   

    ship-state ship-country    B2B order_day order_month  order_weekday  \
0  MAHARASHTRA           IN  

### Feature Engineering

In [11]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')


In [12]:
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from category_encoders.count import CountEncoder

def build_lgbm_preprocessor(num_cols, cat_cols):
    
    # Numerical features: only imputation
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median"))
    ])

    # Categorical features: frequency encoding (LightGBM-friendly)
    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", CountEncoder(normalize=True))
    ])

    return ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])


In [17]:
# for linear models -> scaling is on!
preprocessor = build_lgbm_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
)

In [18]:
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [19]:

# seperating the input and target variable from data
TARGET = "Amount"   # regression target

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [20]:
# fill target with median values
y.fillna(value=data['Amount'].median(), inplace=True)


In [21]:
X.shape

(128975, 16)

In [22]:
y.shape

(128975,)

In [23]:
# seperating the input and target data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [24]:
X_train.shape

(103180, 16)

In [25]:
y_train.shape

(103180,)

In [26]:
# apply preprocessor and take the sample records

def process_and_sample(X_train, X_test, y_train, sample_frac=0.1, n_bins=10, preprocessor=None, random_state=42):
    # 1. Fit preprocessor on full X_train
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # 2. Create quantile bins for stratified sampling
    y_bins = pd.qcut(y_train, q=n_bins, duplicates="drop")
    
    # 3. Stratified sampling
    X_train_sampled, _, y_train_sampled, _ = train_test_split(
        X_train_processed,
        y_train,
        test_size=1-sample_frac,   # keep fraction for training
        stratify=y_bins,
        random_state=random_state
    )
    
    return X_train_processed, X_test_processed, X_train_sampled, y_train_sampled


In [27]:
X_train_processed, X_test_processed, X_train_sampled, y_train_sampled = process_and_sample(
    X_train, X_test, y_train, sample_frac=0.25, preprocessor=preprocessor
)

print("Full X_train_processed shape:", X_train_processed.shape)
print("Sampled X_train_sampled shape:", X_train_sampled.shape)


Full X_train_processed shape: (103180, 16)
Sampled X_train_sampled shape: (25795, 16)


### Model Training

here, we can see that others params are still similar like the single model that we trained, and only the mean absoulte error improved well!

so, this is how the RandomSearchCV finds the best model which is trained with best parameters on passed grid.

In [29]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 1.4 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.5 MB 1.4 MB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.5 MB 1.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.5 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.3 MB/s  0:00:01


a) training with ***sampled data***

In [30]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Build model 
lgbm = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)


In [31]:

# Train on sampled data
lgbm.fit(X_train_sampled, y_train_sampled)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043313 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 531
[LightGBM] [Info] Number of data points in the train set: 25795, number of used features: 14
[LightGBM] [Info] Start training from score 645.229525


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [32]:

# Evaluate on full test set
y_pred = lgbm.predict(X_test_processed)


C:\Users\default.DESKTOP-GAN0M7C\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [33]:

metrics = {
    "MAE": mean_absolute_error(y_test, y_pred),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R2": r2_score(y_test, y_pred)
}

metrics


{'MAE': 87.84672264606058,
 'MSE': 22214.12996131473,
 'RMSE': np.float64(149.0440537603387),
 'R2': 0.7066215824854905}

b) training on ***entire data***

In [34]:
lgbm.fit(X_train_processed, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 549
[LightGBM] [Info] Number of data points in the train set: 103180, number of used features: 15
[LightGBM] [Info] Start training from score 645.056169


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [35]:

# Evaluate on full test set
y_pred = lgbm.predict(X_test_processed)


C:\Users\default.DESKTOP-GAN0M7C\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [36]:

metrics = {
    "MAE": mean_absolute_error(y_test, y_pred),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R2": r2_score(y_test, y_pred)
}

metrics


{'MAE': 82.13630269847556,
 'MSE': 20032.440087462284,
 'RMSE': np.float64(141.53600279597515),
 'R2': 0.7354348073929213}

actually, the model trained with entire data in a very fast way! and the results of sampled trained and entire trained data is very similar.

so, we can fine the best parameters for this LightGBM using the training on complete data.

In [37]:

lgbm_base = LGBMRegressor(
    objective="regression",
    random_state=42,
    n_jobs=-1
)


In [38]:
param_dist = {
    "n_estimators": [300, 500, 800, 1200],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "num_leaves": [31, 63, 127],
    "max_depth": [-1, 5, 8, 12],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9],
    "min_child_samples": [10, 20, 40]
}


In [39]:
from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(
    estimator=lgbm_base,
    param_distributions=param_dist,
    n_iter=40,                  # practical & efficient
    scoring="neg_mean_absolute_error",
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train_sampled, y_train_sampled)


Fitting 3 folds for each of 40 candidates, totalling 120 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005676 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 534
[LightGBM] [Info] Number of data points in the train set: 25795, number of used features: 15
[LightGBM] [Info] Start training from score 645.229525


,estimator,LGBMRegressor...ndom_state=42)
,param_distributions,"{'colsample_bytree': [0.7, 0.8, ...], 'learning_rate': [0.01, 0.03, ...], 'max_depth': [-1, 5, ...], 'min_child_samples': [10, 20, ...], ...}"
,n_iter,40
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,3
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [40]:
best_lgbm = search.best_estimator_
print("Best params:", search.best_params_)


Best params: {'subsample': 0.8, 'num_leaves': 127, 'n_estimators': 1200, 'min_child_samples': 10, 'max_depth': -1, 'learning_rate': 0.03, 'colsample_bytree': 0.7}


In [42]:

y_pred = best_lgbm.predict(X_test_processed)


C:\Users\default.DESKTOP-GAN0M7C\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [43]:


metrics = {
    "MAE": mean_absolute_error(y_test, y_pred),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R2": r2_score(y_test, y_pred)
}

metrics

{'MAE': 82.31454136175925,
 'MSE': 21375.13819425637,
 'RMSE': np.float64(146.2023877857553),
 'R2': 0.7177020109045162}

we can see that the r2_score is improved -> means, the model explains the target variability well!

as this is heavy computational process, so ***choose Sampled data for tuning***.

***LightGBM usually beats XGBoost on speed, matches accuracy***

finally, we can retrain best params on full X_train_processed